# Analysing Conversation logs.

--- 

This was written before the SDK was updated, so I am using requests to pull the logs.

**First you must get your credentials and workspace ID**. You can get all from your workspace of your conversation. 

In [1]:
ctx = {
    'username': 'CONVERSATION_USERNAME',
    'password': 'CONVERSATION_PASSWORD',
    'workspace_id': 'CONVERSATION_WORKSPACE_ID',
    'url': 'https://gateway.watsonplatform.net/conversation/api',
    'version': '2017-04-21'
}

Next lets do our initial setup to make the call. 

In [26]:
import requests
from requests.auth import HTTPBasicAuth
import json

url = '{}/v1/workspaces/{}/logs?version={}'.format(
        ctx.get('url'), 
        ctx.get('workspace_id'), 
        ctx.get('version')
)

basic_auth = HTTPBasicAuth(ctx.get('username'), ctx.get('password'))

Now let's go get the logs from the conversation. It will be in JSON format. 

In [25]:
response = requests.get(url=url, auth=basic_auth)
j = json.loads(response.text)

Let's have a quick look at the JSON and make sure it is well structured.

In [5]:
print(json.dumps(j, indent=2))

removed output for demo (sorry!).


So now we get to play with pandas! This allows us to convert the JSON into a dataframe, which we can manipulate. 

Before we do that, you need to be aware there are normally different kinds of analysis done on logs. Qualitative, Quantitive and Debugging. Each needs different information. 

**Quantitive** - These are fixed metrics, like number of users, response times, common intents, etc.

**Qualitative** - This is analysing how the end user is speaking, and how the system interpreted and responded. Some examples would be where the answer returned may give the wrong impression to the end user. 

**Debugging** - This is really looking for coding issues with your conversation tree. 

For logging purposes these are the main fields. All the information you need is stored in the `j['response']` object array. 

| Field | Usage | Description | 
|:----|:----|:----|
|`input.text`|Qualitative|This is what the user or the application typed in.|
|`intents[]`|Qualitative|This tells you the primary intent for the users question. You should capture the `intent` and `confidence` into columns. If the value is `[]` then means it was irrelevant. |
|`entities[]`|Quantitive|The entities found in relation to the call. With this and intents though, it's important to understand that the application can override these values.|
|`output.text[]`|Qualitative|This is the response shown to the user (or application).|
|`output.log_messages`|Debugging|Capturing this field is handy to look for coding issues within your conversation tree. SPEL errors show up here if they happen.|
|`output.nodes_visited`|Debugging Qualitive |This can be used to see how a progression through a tree happens|
|`context.conversation_id`|All|Use this to group users conversation together. In some solutions however, one pass calls are sometimes done mid conversation. So if you do this, you need to factor that in.|
|`system.branch_exited`|Debugging|This tells you if your conversation left a branch and returned to `root`.|
|`system.branch_exited_reason`|Debugging|If `branch.exited` is `true` then this will tell the why. `completed` means that the branch found a matching node, and finished. `fallback` means that it could not find a matching node, so it jumps back to `root` to find the match.|
|`context.???`|All|You may have context variables you want to capture. You can either do these individually, or code to remove conversation objects and grab what remains|
|`request_timestamp`|Quantitive Qualitative|When conversation received the users response.|
|`response_timestamp`|Quantitive Qualitative|When conversation responded to the user. You can do a delta to see if there are conversation performance issues, but generally keep one of the timestamp fields for analysis.

----

## Using Pandas
For this demo I am only using a couple of fields, but should give you an example on how to build out your own logging. 

In [215]:
import pandas as pd

# Create dataframe.
columns = ['conversation_id', 'request TS', 'response TS', 'User Input', 'Output', 'Intent', 'Confidence', 'Exit Reason', 'Logging']

rows = []

# for object in Json Logs array.
for o in j['logs']:
    row = {}
    
    # Let's shorthand the response and system object.
    r = o['response']
    s = r['context']['system']
        
    row['conversation_id'] = r['context']['conversation_id']
    row['request TS'] = o['request_timestamp']
    row['response TS'] = o['response_timestamp']
    
    if 'text' in r['input']: row['User Input'] = r['input']['text']
    if 'text' in r['output']:row['Output'] = ' '.join(r['output']['text'])
        
    if len(r['intents']) > 0:
        row['Confidence'] = r['intents'][0]['confidence']
        row['Intent'] = r['intents'][0]['intent']

   
    if 'branch_exited_reason' in s: row['Exit Reason'] = s['branch_exited_reason']
    
    if 'log_messaging' in r['output']: row['Logging'] = r['output']['log_messaging']
    
    rows.append(row)

# Build the dataframe. 
df = pd.DataFrame(rows,columns=columns)

# cleaning up dataframe. Removing NaN and converting date fields. 
df = df.fillna('')
df['request TS'] = pd.to_datetime(df['request TS'])
df['response TS'] = pd.to_datetime(df['response TS'])

# Lastly sort by conversation ID, and then request, so that the logs become readable. 
df = df.sort_values(['conversation_id', 'request TS'], ascending=[True, True])

Ok, let's see how that dataframe looks.

In [216]:
df

,conversation_id,request TS,response TS,User Input,Output,Intent,Confidence,Exit Reason,Logging
23,011ad5d4-ecf3-4e3a-8d57-17aafb8c8a30,2017-03-22 09:22:57.398,2017-03-22 09:22:57.440,what breeds do you have?,"I am not fully trained yet, but I believe the ...",DOG_PARENTS_INFORMATION,0.476605,completed,
24,011ad5d4-ecf3-4e3a-8d57-17aafb8c8a30,2017-03-22 09:26:14.596,2017-03-22 09:26:14.620,what is a breed?,"I'm not sure, but this could be the answer. Pa...",DOG_PARENTS_INFORMATION,0.463033,completed,
95,011ad5d4-ecf3-4e3a-8d57-17aafb8c8a30,2017-03-22 11:07:48.484,2017-03-22 11:07:48.509,what breed is best for children/,"I am not sure I understood, but I think this i...",DOG_PARENTS_INFORMATION,0.676506,completed,
96,011ad5d4-ecf3-4e3a-8d57-17aafb8c8a30,2017-03-22 11:07:58.661,2017-03-22 11:07:58.685,what breed is best for children?,Parent information of a dog will be supplied o...,DOG_PARENTS_INFORMATION,0.715889,completed,
90,011ad5d4-ecf3-4e3a-8d57-17aafb8c8a30,2017-03-22 11:14:06.559,2017-03-22 11:14:06.592,how to choose the best breed for kids?,"I'm not sure, but this could be the answer. Ge...",BREEDER_INFORMATION,0.34065,completed,
25,076beabb-22cc-4e76-a4b5-dd5a87af024e,2017-03-22 09:00:46.779,2017-03-22 09:00:46.798,awesome!,I have not been trained on that yet.,BREEDER_INFORMATION,0.07806,completed,
26,076beabb-22cc-4e76-a4b5-dd5a87af024e,2017-03-22 09:00:49.776,2017-03-22 09:00:49.801,ok,I have not been trained on that yet.,PURCHASE_DOG,0.0819037,completed,
28,076beabb-22cc-4e76-a4b5-dd5a87af024e,2017-03-22 09:01:00.967,2017-03-22 09:01:01.030,What kinds of dogs do you sell?,"Get information about us please check our ""Abo...",BREEDER_INFORMATION,0.289015,completed,
27,076beabb-22cc-4e76-a4b5-dd5a87af024e,2017-03-22 09:01:10.701,2017-03-22 09:01:10.730,What's the best dog for kids?,All our dogs eat Brand X Dog food.,DOG_FOOD,0.315675,completed,
55,076beabb-22cc-4e76-a4b5-dd5a87af024e,2017-03-22 09:02:05.759,2017-03-22 09:02:05.778,puppy please,What date do you want to pick up at?,PURCHASE_DOG,0.5795,,


## Examples of Qualitative

In [217]:
# This one gives you how many inputs and outputs per conversation
qualitative1 = df.groupby('conversation_id').count()
qualitative1 = qualitative1.drop(['request TS', 'response TS', 'Intent', 'Confidence', 'Exit Reason', 'Logging'], axis=1)

print('Inputs per conversation:\n    Min: {}.\n    Max: {}.\n    Mean: {}.'.format(
    qualitative1['User Input'].min(),
    qualitative1['User Input'].max(),
    qualitative1['User Input'].mean()
))
print()


# This one will count the intents and sort by decending. 
qualitative2 = df.groupby('Intent').count()
qualitative2 = qualitative2.drop(['request TS', 'response TS', 'User Input', 'Output', 'Confidence', 'Exit Reason', 'Logging'],axis=1)
qualitative2.columns = ['Count']
qualitative2 = qualitative2.sort_values(['Count'], ascending=[False])

qualitative2

Inputs per conversation:
    Min: 1.
    Max: 12.
    Mean: 4.545454545454546.



,Count
Intent,
PURCHASE_DOG,32
BREEDER_INFORMATION,17
,16
DOG_PARENTS_INFORMATION,11
DOG_FOOD,6
ALLOW_VET,3
DOG_SOCIALISATION,3
BREEDER_REFERENCES,2
DOG_HEALTH,2


## Example of Quantitive

These are really determined by reading the logs, and following Quantitive analysis steps. So this is only one example. 

In [221]:
print('This list is an example where conversations misunderstood a question')
quantitive2 = df[df['Output'] == 'I have not been trained on that yet.']
quantitive2 = quantitive2.drop(['request TS', 'response TS', 'Output', 'Exit Reason', 'Logging'], axis=1)

quantitive2

This list is an example where conversations misunderstood a question


,conversation_id,User Input,Intent,Confidence
25,076beabb-22cc-4e76-a4b5-dd5a87af024e,awesome!,BREEDER_INFORMATION,0.07806
26,076beabb-22cc-4e76-a4b5-dd5a87af024e,ok,PURCHASE_DOG,0.0819037
57,076beabb-22cc-4e76-a4b5-dd5a87af024e,See you next week,BREEDER_INFORMATION,0.13978
6,4dafacec-dde1-4580-b0d7-a4ba0a014403,Hi,PURCHASE_DOG,0.0808058
93,90405ca6-7083-4c90-9fb0-a4229a608652,"Ok thanks, bye",ALLOW_VET,0.0915973
85,dc919e4a-0204-4811-8243-611e2ef1e489,Thanks!,ALLOW_VET,0.0826529
33,e276ee6d-bd01-4fb2-a350-48bd49f12efa,What questions can I ask you:,PURCHASE_DOG,0.198781
32,e276ee6d-bd01-4fb2-a350-48bd49f12efa,Hello,PURCHASE_DOG,0.0920247
31,e276ee6d-bd01-4fb2-a350-48bd49f12efa,How old is the business?,BREEDER_INFORMATION,0.190147
61,f5dc320f-ee2a-42ce-bf5d-593f4c98b310,I don't know,DOG_MEET_PARENTS,0.125467


## Examples of Debugging.

In [204]:
print('This list is where you check for possible broken branches')
debug = df[df['Exit Reason'] == 'fallback']
debug

This list is where you check for possible broken branches


,conversation_id,request TS,response TS,User Input,Output,Intent,Confidence,Exit Reason,Logging
61,f5dc320f-ee2a-42ce-bf5d-593f4c98b310,2017-03-22 09:43:31.625,2017-03-22 09:43:31.654,I don't know,I have not been trained on that yet.,DOG_MEET_PARENTS,0.125467,fallback,
